# Data Cleaning

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

In [13]:
train = pd.read_csv('data/train1.csv')

In [14]:
train.head()

,Unnamed: 0,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,ID_279628684,190000.0,0,3,0,1,1,0,0.0,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,Non Vulnerable
1,ID_f29eb3ddd,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,Non Vulnerable
2,ID_68de51c94,ID_68de51c94,0.0,0,8,0,1,1,0,0.0,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,Non Vulnerable
3,ID_d671db89c,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,...,81,289,16,121,4,1.777778,1.0,121.0,289,Non Vulnerable
4,ID_d56d6f5f5,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,Non Vulnerable


In [15]:
train.columns.to_frame()

,0
Unnamed: 0,Unnamed: 0
Id,Id
v2a1,v2a1
hacdor,hacdor
rooms,rooms
...,...
SQBovercrowding,SQBovercrowding
SQBdependency,SQBdependency
SQBmeaned,SQBmeaned
agesq,agesq


## Data Cleaning

Here we create a function to clean the data. We can apply this function to clean both the training set and the test set. 

We filled in missing values for `rez_esc`, `v18q1`, `v2a1`, `meaneduc`, and `SQBmeaned`.

Then we transformed the binary columns into categorical columns related to: 
* housing situation
* education levels
* regions
* relations
* marital
* rubbish location 
* energy source
* toilets
* floor materials
* wall materials
* roof materials
* floor quality
* wall quality
* roof quality
* water provision location
* electricity source

Then recoded the values in each column for easy readability

In [16]:
def data_clean(data):
    #fill in missing values
    data['rez_esc']=data['rez_esc'].fillna(0)
    data['v18q1'] = data['v18q1'].fillna(0)
    v2a1 = data['v2a1'].sort_values()
    med = v2a1.median()
    data.loc[(data['tipovivi1']==1), 'v2a1'] = 0
    data.loc[(data['tipovivi4']==1), 'v2a1'] = med
    data.loc[(data['tipovivi5']==1), 'v2a1'] = med
    meaneduc_nan=data[data['meaneduc'].isnull()][['Id','idhogar','escolari']]
    me=meaneduc_nan.groupby('idhogar')['escolari'].mean().reset_index()
    me
    for row in meaneduc_nan.iterrows():
        idx=row[0]
        idhogar=row[1]['idhogar']
        m=me[me['idhogar']==idhogar]['escolari'].tolist()[0]
        data.at[idx, 'meaneduc']=m
        data.at[idx, 'SQBmeaned']=m*m
        
    #binary columns
    housesitu = ['tipovivi1', 'tipovivi2', 'tipovivi3', 'tipovivi4', 'tipovivi5']
    educlevels = ['instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7',
             'instlevel8', 'instlevel9']
    regions = ['lugar1', 'lugar2', 'lugar3', 'lugar4', 'lugar5', 'lugar6']
    relations = ['parentesco1', 'parentesco2', 'parentesco3', 'parentesco4', 'parentesco5', 'parentesco6',
            'parentesco7', 'parentesco8', 'parentesco9', 'parentesco10', 'parentesco11', 'parentesco12']
    marital = ['estadocivil1', 'estadocivil2', 'estadocivil3', 'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7']
    rubbish = ['elimbasu1', 'elimbasu2', 'elimbasu3', 'elimbasu4', 'elimbasu5', 'elimbasu6']
    energy = ['energcocinar1', 'energcocinar2', 'energcocinar3', 'energcocinar4']
    toilets = ['sanitario1', 'sanitario2', 'sanitario3', 'sanitario5', 'sanitario6']
    floormat = ['pisomoscer', 'pisocemento', 'pisoother', 'pisonatur', 'pisonotiene', 'pisomadera']
    wallmat = ['paredblolad', 'paredzocalo', 'paredpreb', 'pareddes', 'paredmad', 'paredzinc', 'paredfibras', 'paredother']
    roofmat = ['techozinc', 'techoentrepiso', 'techocane', 'techootro']
    floorqual = ['eviv1', 'eviv2', 'eviv3']
    wallqual = ['epared1', 'epared2', 'epared3']
    roofqual = ['etecho1', 'etecho2', 'etecho3']
    waterprov = ['abastaguadentro', 'abastaguafuera', 'abastaguano']
    electric = ['public', 'planpri', 'noelec', 'coopele']
    
    
    #make a dictionary
    binaries = {'housesitu':housesitu,
                'educlevels':educlevels,
                'regions':regions,
                'relations':relations,
                'marital':marital,
                'rubbish':rubbish,
                'energy':energy,
                'toilets':toilets,
                'floormat':floormat,
                'wallmat':wallmat,
                'roofmat':roofmat,
                'floorqual':floorqual,
                'wallqual':wallqual,
                'roofqual':roofqual,
                'waterprov':waterprov,
                'electric':electric
               }
    
    #Replacing the binaries with categorical
    for i in binaries.keys():
        data[i] = data[binaries[i]].idxmax(axis=1)
        data.drop(data[binaries[i]], axis=1, inplace=True)
    
    #recoding values
    hs = {'tipovivi1':'Own', 
      'tipovivi2':'Own/Paying Instllmnts', 
      'tipovivi3':'Rented', 
      'tipovivi4':'Precarious', 
      'tipovivi5':'Other'}
    el = {'instlevel1':'None', 
      'instlevel2':'Incomplete Primary', 
      'instlevel3':'Complete Primary', 
      'instlevel4':'Incomplete Acad. Secondary', 
      'instlevel5':'Complete Acad. Secondary', 
      'instlevel6':'Incomplete Techn. Secondary', 
      'instlevel7':'Complete Techn. Secondary',
      'instlevel8':'Undergrad.', 
      'instlevel9':'Postgrad.'}
    rgn = {'lugar1':'Central', 
       'lugar2':'Chorotega', 
       'lugar3':'Pacafafico Central', 
       'lugar4':'Brunca', 
       'lugar5':'Huetar Atlantica', 
       'lugar6':'Huetar Norte'}
    rltn = {'parentesco1':'Household Head', 
        'parentesco2':'Spouse/Partner', 
        'parentesco3':'Son/Daughter', 
        'parentesco4':'Stepson/Daughter', 
        'parentesco5':'Son/daughter in law', 
        'parentesco6':'Grandson/daughter',
        'parentesco7':'Mother/Father', 
        'parentesco8':'Mother/father in law', 
        'parentesco9':'Brother/sister', 
        'parentesco10':'Brother/sister in law', 
        'parentesco11':'Other family member', 
        'parentesco12':'Other non-family member'}
    mrtl = {'estadocivil1':'< 10 y/o', 
        'estadocivil2':'Free or coupled union', 
        'estadocivil3':'Married', 
        'estadocivil4':'Divorced', 
        'estadocivil5':'Separated', 
        'estadocivil6':'Widow/er', 
        'estadocivil7':'Single'}
    rb = {'elimbasu1':'Tanker Truck', 
      'elimbasu2':'Botan Hollow or Buried', 
      'elimbasu3':'Burning', 
      'elimbasu4':'Thrown in unoccupied space', 
      'elimbasu5':'Thrown in river, creek, or sea', 
      'elimbasu6':'Other'}
    eng = {'energcocinar1':'None', 
       'energcocinar2':'Electricity', 
       'energcocinar3':'Gas', 
       'energcocinar4':'Wood Charcoal'}
    tlt = {'sanitario1':'None', 
       'sanitario2':'Sewer or Cesspool', 
       'sanitario3':'Septic Tank', 
       'sanitario5':'Black hole or letrine', 
       'sanitario6':'Other'}
    flmt = {'pisomoscer':'Mosaic, Ceramic', 
        'pisocemento':'Cement', 
        'pisoother':'Other', 
        'pisonatur':'Natural', 
        'pisonotiene':'None', 
        'pisomadera':'Wood'}
    wlmt = {'paredblolad':'Block/Brick', 
        'paredzocalo':'Socket (wood, zinc, absbesto)', 
        'paredpreb':'Prefabricated/cement', 
        'pareddes':'Waste', 
        'paredmad':'Wood', 
        'paredzinc':'Zinc', 
        'paredfibras':'Natural Fibers', 
        'paredother':'Other'}
    rfmt = {'techozinc':'Metal foil/Zinc', 
        'techoentrepiso':'Fiber cement', 
        'techocane':'Natural fibers', 
        'techootro':'Other'}
    flql = {'eviv1':'Bad', 
        'eviv2':'Regular', 
        'eviv3':'Good'}
    wlql = {'epared1':'Bad',
        'epared2':'Regular', 
        'epared3':'Good'}
    rfqu = {'etecho1':'Bad', 
        'etecho2':'Regular', 
        'etecho3':'Good'}
    wtrpr = {'abastaguadentro':'Inside', 
         'abastaguafuera':'Outside', 
         'abastaguano':'None'}
    elct = {'public':'Public', 
        'planpri':'Private Plant', 
        'noelec':'None', 
        'coopele':'Cooperative'}
    
    #replacing
    data.replace(dict(housesitu=hs, 
                  educlevels=el,
                  regions=rgn,
                  relations=rltn,
                  marital=mrtl,
                  rubbish=rb,
                  energy=eng,
                  toilets=tlt,
                  floormat=flmt,
                  wallmat=wlmt,
                  roofmat=rfmt,
                  floorqual=flql,
                  wallqual=wlql,
                  roofqual=rfqu,
                  waterprov=wtrpr,
                  electric=elct), inplace=True)
    
    return data

### *Clean Training Dataset*

In [17]:
train = pd.read_csv('data/train.csv')
cleaned = data_clean(train)
cleaned.to_csv('data/trainclean.csv', index=False)
cleaned.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,energy,toilets,floormat,wallmat,roofmat,floorqual,wallqual,roofqual,waterprov,electric
0,ID_279628684,190000.0,0,3,0,1,1,0,0.0,0,...,Gas,Sewer or Cesspool,"Mosaic, Ceramic",Block/Brick,Fiber cement,Bad,Regular,Bad,Inside,Public
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,Electricity,Sewer or Cesspool,Wood,Wood,Metal foil/Zinc,Regular,Regular,Regular,Inside,Public
2,ID_68de51c94,0.0,0,8,0,1,1,0,0.0,0,...,Electricity,Sewer or Cesspool,"Mosaic, Ceramic",Wood,Metal foil/Zinc,Good,Regular,Good,Inside,Public
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,Electricity,Sewer or Cesspool,"Mosaic, Ceramic",Block/Brick,Metal foil/Zinc,Good,Good,Good,Inside,Public
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,Electricity,Sewer or Cesspool,"Mosaic, Ceramic",Block/Brick,Metal foil/Zinc,Good,Good,Good,Inside,Public


143 -> 71 columns

In [18]:
train.columns

Index(['Id', 'v2a1', 'hacdor', 'rooms', 'hacapo', 'v14a', 'refrig', 'v18q',
       'v18q1', 'r4h1', 'r4h2', 'r4h3', 'r4m1', 'r4m2', 'r4m3', 'r4t1', 'r4t2',
       'r4t3', 'tamhog', 'tamviv', 'escolari', 'rez_esc', 'hhsize',
       'cielorazo', 'dis', 'male', 'female', 'idhogar', 'hogar_nin',
       'hogar_adul', 'hogar_mayor', 'hogar_total', 'dependency', 'edjefe',
       'edjefa', 'meaneduc', 'bedrooms', 'overcrowding', 'computer',
       'television', 'mobilephone', 'qmobilephone', 'area1', 'area2', 'age',
       'SQBescolari', 'SQBage', 'SQBhogar_total', 'SQBedjefe', 'SQBhogar_nin',
       'SQBovercrowding', 'SQBdependency', 'SQBmeaned', 'agesq', 'Target',
       'housesitu', 'educlevels', 'regions', 'relations', 'marital', 'rubbish',
       'energy', 'toilets', 'floormat', 'wallmat', 'roofmat', 'floorqual',
       'wallqual', 'roofqual', 'waterprov', 'electric'],
      dtype='object')

### *Clean Test Dataset*

In [19]:
test = pd.read_csv('data/test.csv')
clean = data_clean(test)
clean.to_csv('data/testclean.csv', index=False)
clean.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,energy,toilets,floormat,wallmat,roofmat,floorqual,wallqual,roofqual,waterprov,electric
0,ID_2f6873615,0.0,0,5,0,1,1,0,0.0,1,...,Gas,Sewer or Cesspool,"Mosaic, Ceramic",Block/Brick,Metal foil/Zinc,Good,Good,Good,Inside,Public
1,ID_1c78846d2,0.0,0,5,0,1,1,0,0.0,1,...,Gas,Sewer or Cesspool,"Mosaic, Ceramic",Block/Brick,Metal foil/Zinc,Good,Good,Good,Inside,Public
2,ID_e5442cf6a,0.0,0,5,0,1,1,0,0.0,1,...,Gas,Sewer or Cesspool,"Mosaic, Ceramic",Block/Brick,Metal foil/Zinc,Good,Good,Good,Inside,Public
3,ID_a8db26a79,0.0,0,14,0,1,1,1,1.0,0,...,Electricity,Sewer or Cesspool,"Mosaic, Ceramic",Block/Brick,Metal foil/Zinc,Good,Good,Good,Inside,Public
4,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,0,...,Electricity,Sewer or Cesspool,"Mosaic, Ceramic",Block/Brick,Metal foil/Zinc,Good,Regular,Good,Inside,Public
